In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging

logger = logging.getLogger("nlstruct")
logger.setLevel(logging.DEBUG)


In [3]:
import string
import logging
import pandas as pd

import sys
sys.path.insert(0,'/home/ytaille/deep_multilingual_normalization')

from nlstruct.utils import torch_clone
from nlstruct.utils import torch_global as tg
from deep_multilingual_normalization.preprocess import preprocess, load_quaero
from deep_multilingual_normalization.train import train_step1, train_step2, clear
from deep_multilingual_normalization.eval import compute_scores, predict

from transformers import AutoModel
from deep_multilingual_normalization.model import Classifier, FastClusteredIPSearch

from notebook_utils import *

In [4]:
from create_classifiers import create_classifiers

classifier, classifier2 = create_classifiers()

Available CUDA devices: 1
Current device: cuda:0
Using cache /home/ytaille/data/cache/preprocess_training_data/579ec808c912b49f
Loading /home/ytaille/data/cache/preprocess_training_data/579ec808c912b49f/output.pkl... 
Using cache /home/ytaille/data/cache/ncbi_raw_files/4d8c0405832b0f7e
Quaero mentions: 5259
Total deduplicated synonyms: 18123
Total deduplicated labels: 13051
Quaero mentions: 16283
Normalized split, with given vocabulary and no unk
Normalized label, with given vocabulary and no unk
Normalized quaero_source, with given vocabulary and no unk
Normalized token, with given vocabulary and no unk
Normalized group, with given vocabulary and no unk
Normalized source, with given vocabulary and no unk
Using cache /home/ytaille/data/cache/norm/paper/train_step1/34ef6f8317449a23
Loading /home/ytaille/data/cache/norm/paper/train_step1/34ef6f8317449a23/history.yaml... 
epoch | train_loss | train_acc | val_loss | val_acc | val_map | acc_emea | acc_medline |        lr |   norm |     step

In [5]:
bert_name = "bert-base-multilingual-uncased"

dataset = load_from_brat("/home/ytaille/data/tmp/ws_inputs")

docs, sentences, tokens, deltas, vocs = preprocess(
    dataset=dataset,
    max_sentence_length=120,
    bert_name=bert_name,
    vocabularies=None,
)

prep = Dataset(
    sentences=sentences,
    tokens=tokens,
    deltas=deltas,
)

batcher, encoded, ids = make_batcher(docs, sentences, tokens)

batch_size = 32
with_tqdm = True
with_groups = False
topk = 1
save_embeds = False

tg.set_device('cuda:0')
device = tg.device

with evaluating(classifier):  # eval mode: no dropout, frozen batch norm, etc
    with torch.no_grad():  # no gradients -> faster
        with tqdm(batcher.dataloader(batch_size=batch_size, device=device, sparse_sort_on="token_mask"), disable=not with_tqdm, mininterval=10.) as bar:
            for batch in bar:
                res = classifier2.forward(
                    tokens=batch["token"],
                    mask=batch["token_mask"],
                    groups=batch["group"] if with_groups else None,
                    return_scores=topk > 0.1,
                    return_embeds=save_embeds,
                )

/home/ytaille/.conda/envs/deep_mult_norm/lib/python3.7/site-packages/nlstruct/text/chunking/huggingface.py:11: FutureWarning: doc_id_col is not used anymore in the huggingface_tokenize function
  warnings.warn("doc_id_col is not used anymore in the huggingface_tokenize function", FutureWarning)


Available CUDA devices: 1
Current device: cuda:0


  0%|          | 0/1 [00:00<?, ?it/s]


AttributeError: 'NoneType' object has no attribute 'forward'

In [6]:
best_probs = res['scores'].topk(dim=-1, k=topk)

In [7]:
best_probs

torch.return_types.topk(
values=tensor([[8.3149],
        [9.0723],
        [7.5563],
        [8.5602]], device='cuda:0'),
indices=tensor([[528324],
        [488454],
        [ 98788],
        [192771]], device='cuda:0'))

In [8]:
best_probs.values

tensor([[8.3149],
        [9.0723],
        [7.5563],
        [8.5602]], device='cuda:0')